In [1]:
%reload_ext autoreload
%autoreload 2

from cleaning_funcs import *

import numpy as np
import pandas as pd

# Preprocessing Script
The data underlying the results presented in the study are available upon request, with consent from each individual lab that contributed data to the study before sharing their data. This notebook will not run without the data but is still included to show how we prepared the standardized lab dataset for our analysis. 

The final cleaned and processed dataset produced at the end of this notebook is included in this Github repository.

# Prepare Dataset

## Load Dataset

In [2]:
lab_data = pd.read_csv('data/standardized_lab_data_220218.csv', parse_dates=['test_date'])
df = lab_data.copy()

/home/christiana/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# indicate lab in producer name
df['anon_producer2'] = df['anon_producer'].astype(str).str.extract('(\d+)') 
df['producer_id'] = df['anon_producer2']+'_'+df['lab_id']

# get unknown producers
prod_bool = df['anon_producer2'].isnull()

# treat producers as separate across data dumps for CA
lab_bool = df['lab_id']=='confidence_analytics'
df.loc[lab_bool, 'producer_id'] = df.loc[lab_bool, 'producer_id'] + df.loc[lab_bool, 'dump_no'].astype(str)

# further anonymize IDs
df['anon_producer'] = df['producer_id'].astype('category').cat.codes.astype('str')
df.loc[prod_bool, 'anon_producer'] = np.nan

## Cannab Data Prep

In [4]:
# fill acid rows, also minor cannabs missing acids
cannabs_0 = ['thca','cbda','cbga','thcva']
cannabs_missing_a = ['cbca','cbdva','cbna','cbta','d8_thca']
df[cannabs_0] = df[cannabs_0].fillna(0)
df[cannabs_missing_a] = 0

# designate pairs
cannab_pairs = [('thc','thca'),
               ('cbd','cbda'),
               ('cbg','cbga'),
               ('cbc','cbca'),
               ('cbn','cbna'),
               ('cbt','cbta'),
               ('d8_thc','d8_thca')]
cannab_pairs_v = [('thcv','thcva'),
                 ('cbdv','cbdva')]

# decarb cannabs
for cannab_pair in cannab_pairs:
    col_name = 'tot_'+cannab_pair[0]
    df[col_name] = decarb(df, cannab_pair[0], cannab_pair[1])

# decarb cannabs -varin
for cannab_pair in cannab_pairs_v:
    col_name = 'tot_'+cannab_pair[0]
    df[col_name] = decarb(df, cannab_pair[0], cannab_pair[1], varin=True)
    
cannab_cols = ['tot_thc',
'tot_cbd',
'tot_cbg',
'tot_cbc',
'tot_cbn',
'tot_cbdv',
'tot_cbt',
'tot_d8_thc',
'tot_thcv',
'cbc','cbca',
'cbd','cbda',
'cbdv','cbdva',
'cbg','cbga',
'cbn','cbna',
'cbt','cbta',
'd8_thc','d8_thca',
'thc','thca',
'thcv','thcva']

tot_cannab_cols = ['tot_thc',
'tot_cbd',
'tot_cbg',
'tot_cbc',
'tot_cbn',
'tot_cbdv',
'tot_cbt',
'tot_d8_thc',
'tot_thcv']

In [5]:
# determine THC/CBD ratio
df['chemotype_ratio'] = df['tot_thc'].div(df['tot_cbd'], fill_value=0)
df.loc[(df['tot_thc']==0)&(df['tot_cbd']!=0), 'chemotype_ratio'] = -np.inf
df.loc[(df['tot_thc']!=0)&(df['tot_cbd']==0), 'chemotype_ratio'] = np.inf

# bin chemotypes by ratio
df['chemotype'] = pd.cut(df['chemotype_ratio'],
                            [-np.inf, 0.2, 5, np.inf],
                             labels=['CBD-Dom','Bal THC/CBD', 'THC-Dom'],
                        include_lowest=True)

In [6]:
# create cannab bool
df['has_cannabs'] = ~df['chemotype'].isnull()

## Terp Data Prep

In [7]:
terps = ['a_cedrene', 'a_ocimene', 'a_phellandrene', 'a_pinene', 'a_terpinene',
       'a_terpineol', 'b_maaliene', 'b_nerolidol', 'b_ocimene', 'b_pinene',
       'bisabolol', 'borneol', 'borneol_isomers', 'camphene', 'camphor',
       'carene', 'caryophyllene', 'caryophyllene_oxide', 'cedrol',
       'cis_nerolidol', 'cis_ocimene', 'cis_phytol', 'citronellol',
       'eucalyptol', 'farnesene', 'fenchol', 'fenchone', 'fenchyl_alcohol',
       'g_terpinene', 'g_terpineol', 'geraniol', 'geranyl_acetate', 'guaiol',
       'humulene', 'iso_borneol', 'iso_pulegol', 'limonene', 'linalool',
       'menthol', 'myrcene', 'nerol', 'nerolidol', 'ocimene', 'p_cymene',
       'phytol', 'pulegone', 'sabinene', 'sabinene_hydrate', 'selinadiene',
       'terpineol', 'terpinolene', 'thujene', 'thymol', 'trans_nerolidol',
       'trans_ocimene', 'trans_phytol', 'valencene']

# drop empty terp columns
empty_terps = df[terps].columns[(~df[terps].isnull()).sum()==0]
print('n terp cols dropped: '+str(len(empty_terps)))
df = df.drop(columns=empty_terps).copy()
terps = [x for x in terps if x not in empty_terps]

n terp cols dropped: 4


In [8]:
# update ChemHistory measurements for nerolidol and farnesene
chem_bool = df['lab_id']=='chemhistory'
date_bool = df['test_date']<'2020-04-01'

df.loc[chem_bool & date_bool,
       ['cis_nerolidol','trans_nerolidol','nerolidol','farnesene']] = np.nan

In [9]:
# count total non-zero terps for each sample
df['n_measured_terps'] = (~df[terps].isnull()).sum(axis=1)

# create terp bool
terp_bool = df['n_measured_terps']>0
df['has_terps'] = terp_bool

df['has_both'] = df['has_cannabs']&df['has_terps']

In [10]:
# specify nerolidol isomers
nero = ['cis_nerolidol','nerolidol','trans_nerolidol']
df['tot_nerolidol_ct'] = df[nero].sum(axis=1)
terps.append('tot_nerolidol_ct')

# specify ocimene isomers
oci = [x for x in terps if 'ocimene' in x]
df['tot_ocimene'] = df[oci].sum(axis=1)
terps.append('tot_ocimene')

# update terp list
old_terps = terps.copy()
terps = [x for x in terps if x not in oci+nero]

# get common terps
lab_terp_counts = df[df['has_terps']].groupby('lab_id')[terps].apply(lambda x: (~pd.isnull(x)).sum()>0).sum().sort_values(ascending=False)

common_terps = (lab_terp_counts.head(15).index).drop('iso_pulegol')

print(common_terps)

Index(['tot_ocimene', 'camphene', 'g_terpinene', 'tot_nerolidol_ct',
       'humulene', 'caryophyllene', 'limonene', 'linalool', 'myrcene',
       'bisabolol', 'b_pinene', 'a_terpinene', 'terpinolene', 'a_pinene'],
      dtype='object')


In [11]:
# get count of common terps which == 0
df['n_zero_terps'] = (df[common_terps]==0).sum(axis=1)

# get top terp and top terp (common) for each sample
df['top_terp'] = np.nan
df['top_terp_f'] = np.nan
terp_dict = {terp: (terp if terp in common_terps else 'other') for terp in terps}
df.loc[df['has_terps'], 'top_terp'] = df.loc[df['has_terps'], terps].apply(lambda x: x.idxmax(), axis=1)
df.loc[df['has_terps'], 'top_terp_f'] = df.loc[df['has_terps'], terps].apply(lambda x: terp_dict[x.idxmax()], axis=1)

# get total terps
df['total_terps'] = df.loc[:, terps].sum(axis=1)

# calculate terp variance
df['terp_var'] = df[terps].apply(lambda x: np.var(x), axis=1)

# Initial Cleaning

In [12]:
# here are the starting ns for each lab
pre_ns = df.groupby('lab_id')[['has_cannabs','has_both']].sum()
pre_ns.loc['total'] = pre_ns.sum(axis=0)
pre_ns

,has_cannabs,has_both
lab_id,,
lab0,6255,6202
lab1,13546,13544
lab2,53233,11138
lab3,1620,886
lab4,7243,7241
lab5,8119,8119
total,90016,47130


In [13]:
def unique_count(x):
    return x.dropna().unique().shape[0]

df.groupby('lab_id')[['anon_producer','strain_slug']].agg({'anon_producer':unique_count, 'strain_slug':unique_count})

,anon_producer,strain_slug
lab_id,,
lab0,296,836
lab1,591,1539
lab2,688,1800
lab3,5,121
lab4,544,749
lab5,1060,1218


In [14]:

df[['anon_producer','strain_slug']].agg({'anon_producer':unique_count, 'strain_slug':unique_count})

anon_producer    3184
strain_slug      3090
dtype: int64

In [15]:
remove = df.duplicated(subset=[x for x in df.columns if x not in 'strain_slug']).copy()
df = print_remove(df, remove)

n samples dropped = 11


# Cannab Cleaning

In [16]:
# hard filter: remove any samples with cannab over thresh
cannab_max = 40
remove = (df.loc[:, cannab_cols]>cannab_max).sum(axis=1).astype(bool)
df = print_remove(df, remove)

n samples dropped = 80


In [17]:
# remove any samples with summed cannabs over thresh
cannab_tot_max = 50
remove = df[tot_cannab_cols].sum(axis=1)>cannab_tot_max
df = print_remove(df, remove)

n samples dropped = 2


In [18]:
# get rid of data that doesn't have a chemotype bc of null values

remove = df['chemotype'].isnull()
df = print_remove(df, remove)

n samples dropped = 591


# Terp Cleaning

In [19]:
# get terp var thresh
terp_var_thresh = 0.001
terp_var_bool = (df.loc[:, 'terp_var']<=terp_var_thresh)
remove = terp_var_bool&df['has_terps']
df = print_remove(df, remove, terp=True, terps=terps)

n samples dropped = 2041


In [20]:
# remove terp data for samples  w/ a terp measure over threshold
terp_max = 5
remove = (df.loc[:, terps]>terp_max).sum(axis=1).astype(bool)
df = print_remove(df, remove, terp=True, terps=terps)

n samples dropped = 8


In [21]:
# remove terp data for samples w/ more than n common terps measuring zero
n_zero_max = 10
n_zero_bool = df['n_zero_terps'] >= n_zero_max
remove = n_zero_bool&df['has_terps']
df = print_remove(df, remove, terp=True, terps=terps)

n samples dropped = 1880


# Final Assembly

In [22]:
# here are the final ns for each lab
post_ns = df.groupby('lab_id')[['has_cannabs','has_terps']].sum()
post_ns.loc['total'] = post_ns.sum(axis=0)
post_ns

,has_cannabs,has_terps
lab_id,,
lab0,6253,6173
lab1,13508,12025
lab2,53190,11070
lab3,1620,695
lab4,7240,5268
lab5,8112,7917
total,89923,43148


In [23]:
df.groupby('lab_id')[['anon_producer','strain_slug']].agg({'anon_producer':unique_count, 'strain_slug':unique_count})

,anon_producer,strain_slug
lab_id,,
lab0,293,834
lab1,589,1538
lab2,672,1794
lab3,5,121
lab4,543,748
lab5,1058,1218


In [24]:

df[['anon_producer','strain_slug']].agg({'anon_producer':unique_count, 'strain_slug':unique_count})

anon_producer    3160
strain_slug      3087
dtype: int64

In [25]:
unique_count(df['strain_slug'])

3087

In [26]:
# create unique id
df = df.reset_index(drop=True)
df['u_id'] = df.index

In [27]:
# add in Leafly strain info
strain_info = pd.read_csv('data/strain_page_info_20210325.csv')
strain_info.loc[pd.isnull(strain_info['strain_category']), 'strain_category'] = 'None'

# save strain info csv
strain_info[['strain_slug','strain_category','strain_popularity']].to_csv('data/strain_info_pub_20210915.csv', index=False)

In [28]:
df = df.merge(strain_info[['strain_slug','strain_category','strain_popularity']], on='strain_slug', how='left')
df['strain_category'] = df['strain_category'].fillna('None')

In [29]:
info_cols = ['u_id',
             'lab_id',
             'strain_slug',
             'anon_producer',
             'region',
             'product_category',
             'strain_category',
             'strain_popularity',
             'chemotype']

cannab_info_cols = ['has_cannabs','chemotype_ratio']

terp_info_cols = ['has_terps',
                 'total_terps',
                 'terp_var',
                 'top_terp_f']

cannab_cols = ['tot_thc',
                'tot_cbd',
                'tot_cbg',
                'tot_cbc',
                'tot_cbn',
                'tot_thcv']

# terp_cols = list(common_terps)+[x for x in old_terps if x not in common_terps]
terp_cols = list(common_terps)

In [30]:
# reorder columns just for neatness
fin_df = df[info_cols+cannab_info_cols+cannab_cols+terp_info_cols+terp_cols]
fin_df.to_csv('data/preproc_lab_data_pub_20220218.csv', index=False)